In [3]:
!python DPO_trainer.py
# !python SFT_trainer.py
# !python SALT_trainer.py

# TODO adapt the codes to print metrics

In [6]:
# from metrics import AutomaticNgramEval, Rouge, AutomaticFactEval

# ngram_eval = AutomaticNgramEval()
# # factev = AutomaticFactEval()

In [8]:
# global eval_output_record
# eval_output_record = {}


In [ ]:
# for key, df in eval_output_record.items():
#     df.to_csv(script_args.output_dir + '/' + str(key) + ".csv", index=False)